# Unsupervised ML on the Descartes Labs Platform: Interactive Deployment with Dynamic Compute
__________________

This notebook will demonstrate a typical example of how to interact with the results of a machine learning model using Descartes Labs Platform APIs and define new AOIs to analyze on-the-fly. 

The general steps covered in this notebook are:
* Retrieve a running [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function)
* Display results overlain on input imagery in a web map with [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html)
* Specify new areas to apply our model over with interactive [widgets](https://ipywidgets.readthedocs.io/en/stable/)

_Note_: In order to run this example you must first complete the steps outlined in both [01a Training an Unsupervised Classifier.ipynb](01a%20Training%20an%20Unsupervised%20Classifier.ipynb) and [01b Deploying an Unsupervised Classifier.ipynb](01b%20Deploying%20an%20Unsupervised%20Classifier.ipynb).

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import properties as p
import descarteslabs.dynamic_compute as dc

from descarteslabs.compute import Function
from descarteslabs.dynamic_compute import ImageStack, Mosaic

In [ ]:
import geopandas as gpd
from datetime import datetime
from ipyleaflet import DrawControl

Defining global variables for reference throughout this example, including the current user's ID:

In [ ]:
user_hash = dl.auth.Auth().namespace
org = dl.auth.Auth().payload['org']

And associated product IDs and bands:

In [ ]:
kmeans_pid = f"{org or user_hash}:kmeans-results-{user_hash}"
kmeans_pid

## Retrieving an Active Compute Function 

If you lost your ID, you can retrieve it at [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) or search the latest created Function with that name as below:

In [ ]:
func_search = (
    Function.search()
    .filter(p.owner == user_hash)
    .filter(p.name.startswith("Run KMeans Model Inference"))
    .sort(-Function.creation_date)
    .limit(1)
).collect()

for func in func_search:
    print(func.id)
    print(func.creation_date)

In [ ]:
async_func = func_search[0]
async_func

## Setting Up Dynamic Compute

Next we will set  up the interactive map components. First create a Sentinel-2 [`ImageStack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack) for our time period:

In [ ]:
s2_stack = ImageStack.from_product_bands(
    s2_pid,
    bands,
    start_datetime="2023-06-01",
    end_datetime="2023-09-01",
).filter(lambda x: x.cloud_fraction < 0.1)

Next we'll declare the map alongside center coordinates and zoom level:

In [ ]:
m = dc.map
m.center = 44.4729, -73.1657
m.zoom = 13

Now visualize our Sentinel-2 data on the map:

In [ ]:
s2_stack.mean(axis="images").visualize("Sentinel-2", m)

Next define a simple [`DrawControl`](https://ipyleaflet.readthedocs.io/en/latest/controls/draw_control.html) widget:

In [ ]:
# This is some interactivity with the map we'll embed below:
draw_control = DrawControl()
# Drawn polygon styling
draw_control.polygon = {
    "shapeOptions": {"fillColor": "green", "color": "blue", "fillOpacity": 0.5},
    "drawError": {"color": "red", "message": "Oops!"},
    "allowIntersection": False,
}

# Setting empty feature collection to track as we draw polys:
feature_collection = {"type": "FeatureCollection", "features": []}

# Define this handle_draw function for the Draw Control widget
def handle_draw(target, action, geo_json):
    # Clears feature collection on each new polygon with new geojson
    feature_collection["features"] = [geo_json]


# Adding the handle_draw function to the Draw Control widget
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

**_Note on Updating Tile Layers_**

Our results [`Mosaic`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic) object will "refresh" it's tile layers upon re-instantiation of its class, as shown in the cell below. 

If you are waiting for your function to process in real time you will need to re-run the following cell to update your imagery as each job completes:

In [ ]:
kmeans_mosaic = Mosaic.from_product_bands(kmeans_pid, "class")
kmeans_mosaic.visualize("KMeans Results", m, colormap="terrain")

and instantiate our map:

In [ ]:
m

## Interacting with the Results

Notice that we now have the option embedded in our mapframe to draw new polygons. When you complete a new polygon on the map, run the following cell to format a new list of arguments to pass to the currently running asynchronous function:

In [ ]:
drawn_gdf = gpd.GeoDataFrame.from_features(feature_collection, crs=4326)
# Create a new set of DLTiles for this new AOI
geocontext_geom = drawn_gdf["geometry"][0]
# You could also pass the map's geocontext as a WKT
# geocontext_geom = box(*wf.map.geocontext().bounds)
dltiles = dl.geo.DLTile.from_shape(
    geocontext_geom, resolution=10.0, tilesize=2048, pad=0
)
args = [(dltile.key, kmeans_pid) for dltile in dltiles]
len(args)

Lastly, we submit those arguments to our running function to process:

In [ ]:
jobs = async_func.map(args)
len(jobs)